In [28]:

!pip install ffmpeg-python


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import sys
import time
from pydub import AudioSegment
import soundfile as sf
import numpy as np
import librosa
from pyAudioAnalysis import audioSegmentation as aS
from pytube import YouTube

def main(argv):
    if argv[1] == "-shortTerm":
        audio_file = "diarizationExample.wav"
        process_short_term(audio_file)
    elif argv[1] == "-classifyFile":
        audio_file = "diarizationExample.wav"
        process_classify_file(audio_file)
    elif argv[1] == "-mtClassify":
        audio_file = "diarizationExample.wav"
        process_mt_classification(audio_file)
    elif argv[1] == "-hmmSegmentation":
        audio_file = "diarizationExample.wav"
        process_hmm_segmentation(audio_file)
    elif argv[1] == "-silenceRemoval":
        audio_file = "diarizationExample.wav"
        process_silence_removal(audio_file)
    elif argv[1] == "-thumbnailing":
        audio_file = "scottish.wav"
        process_thumbnailing(audio_file)
    elif argv[1] == "-diarization-noLDA":
        audio_file = "diarizationExample.wav"
        process_diarization_no_lda(audio_file)
    elif argv[1] == "-diarization-LDA":
        audio_file = "diarizationExample.wav"
        process_diarization_lda(audio_file)

def process_short_term(audio_file):
    try:
        y, sr = sf.read(audio_file)
        duration = librosa.get_duration(y=y, sr=sr)
        t1 = time.process_time()
        # Perform audio feature extraction using librosa
        # Replace the following line with your specific feature extraction code
        F = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(0.050 * sr), n_fft=int(0.050 * sr))
        t2 = time.process_time()
        per_time = duration / (t2 - t1)
        print("Short-term feature extraction: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during short-term feature extraction:", e)

def process_classify_file(audio_file):
    try:
        t1 = time.process_time()
        # Perform file classification using pyAudioAnalysis
        aS.file_classification(audio_file, "svmSM", "svm")
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Mid-term feature extraction + classification: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during mid-term feature extraction + classification:", e)

def process_mt_classification(audio_file):
    try:
        t1 = time.process_time()
        # Perform fix-sized classification - segmentation using pyAudioAnalysis
        _, _, acc = aS.mt_file_classification(audio_file, "svmSM", "svm", False, '')
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Fix-sized classification - segmentation: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during fix-sized classification - segmentation:", e)

def process_hmm_segmentation(audio_file):
    try:
        t1 = time.process_time()
        # Perform HMM-based classification - segmentation using pyAudioAnalysis
        aS.hmm_segmentation(audio_file, 'hmmRadioSM', False, '')
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("HMM-based classification - segmentation: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during HMM-based classification - segmentation:", e)

def process_silence_removal(audio_file):
    try:
        t1 = time.process_time()
        audio = AudioSegment.from_file(audio_file)
        samples = audio.get_array_of_samples()
        sample_rate = audio.frame_rate
        # Perform silence removal using pyAudioAnalysis
        segments = aS.silence_removal(samples, sample_rate, 0.050, 0.050, smooth_window=1.0, weight=0.3, plot=False)
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Silence removal: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during silence removal:", e)

def process_thumbnailing(audio_file):
    try:
        t1 = time.process_time()
        audio = AudioSegment.from_file(audio_file)
        samples = audio.get_array_of_samples()
        sample_rate = audio.frame_rate
        # Perform thumbnailing using pyAudioAnalysis
        A1, A2, B1, B2, Smatrix = aS.music_thumbnailing(samples, sample_rate, 1.0, 1.0, 15.0)
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Thumbnailing: {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during thumbnailing:", e)

def process_diarization_no_lda(audio_file):
    try:
        t1 = time.process_time()
        # Perform speaker diarization (no LDA) using pyAudioAnalysis
        aS.speaker_diarization(audio_file, 4, LDAdim=0, PLOT=False)
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Diarization (no LDA): {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during diarization (no LDA):", e)

def process_diarization_lda(audio_file):
    try:
        t1 = time.process_time()
        # Perform speaker diarization (LDA) using pyAudioAnalysis
        aS.speaker_diarization(audio_file, 4, PLOT=False)
        t2 = time.process_time()
        per_time = get_audio_duration(audio_file) / (t2 - t1)
        print("Diarization (LDA): {:.1f}x realtime".format(per_time))
    except Exception as e:
        print("Error occurred during diarization (LDA):", e)

def get_audio_duration(audio_file):
    audio = AudioSegment.from_file(audio_file)
    return len(audio) / 1000  # duration in seconds

def extract_audio_from_youtube(video_id, output_file):
    yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")
    audio = yt.streams.filter(only_audio=True).first()
    audio.download(output_path="", filename=output_file)

def main():
    audio_file = r"C:\Users\nites\Downloads\ML Project\Subtitles generation\audio.wav"
    functions = [process_short_term, process_classify_file]  # Add other functions to this list
    for function in functions:
        try:
            function(audio_file)
        except Exception as e:
            print(f"Error occurred during {function.__name__}:", e)
    print()  # Print a blank line after each function execution

if __name__ == '__main__':
    main()

C:\Users\nites\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Error occurred during short-term feature extraction: Error opening 'C:\\Users\\nites\\Downloads\\ML Project\\Subtitles generation\\audio.wav': System error.
Error occurred during mid-term feature extraction + classification: module 'pyAudioAnalysis.audioSegmentation' has no attribute 'file_classification'



C:\Users\nites\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\nites\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified